# setup
https://github.com/pawelc/NeuralLikelihoods

In [1]:
!git clone https://github.com/pawelc/NeuralLikelihoods.git chilinski2020neural
%cd chilinski2020neural

Cloning into 'chilinski2020neural'...
remote: Enumerating objects: 468, done.
remote: Counting objects: 100% (468/468), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 468 (delta 264), reused 398 (delta 195), pack-reused 0
Receiving objects: 100% (468/468), 11.85 MiB | 7.99 MiB/s, done.
Resolving deltas: 100% (264/264), done.
/content/chilinski2020neural


In [2]:
import sys
sys.path.append('code')

In [3]:
%load_ext autoreload
%autoreload 2

import os
import json
from functools import reduce
import numpy as np

from conf import conf

import tensorflow as tf
tfk = tf.keras
import tensorflow_probability as tfp
tfd = tfp.distributions
import tensorflow_datasets as tfds
K = tfk.backend

import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import iplot

from models.tensorflow.rnade import Rnade
from models.tensorflow.rnade_deep import RnadeDeep
from models.tensorflow.maf import MAF
from models.tensorflow.mdn import MDN
from models.tensorflow.monde import MONDE
from models.tensorflow.monde_ar_made import MondeARMADE
from data import registry
from models.tensorflow.compute import get_device
from utils import elapsed_timer
import utils as nl_utils
from scipy.stats import bootstrap

# data
https://github.com/yutingng/gen-AC.git

In [4]:
import pandas as pd
import scipy
import copy

from sklearn.model_selection import train_test_split

In [5]:
def add_train_random_noise(data, num_adds):
  new_data = np.random.rand(num_adds, data.shape[1])
  return np.concatenate((data, new_data), axis = 0)

In [6]:
def rank_normalization(X):
  X = copy.deepcopy(X)
  for z in X:
      ndata = z.shape[0]
      gap = 1./(ndata+1)
      nfeats = z.shape[1]
      for i in range(nfeats):
          z[:, i] = scipy.stats.rankdata(z[:, i], 'ordinal')*gap
  return X

In [7]:
!git clone https://github.com/yutingng/gen-AC.git

Cloning into 'gen-AC'...
remote: Enumerating objects: 466, done.
remote: Counting objects: 100% (466/466), done.
remote: Compressing objects: 100% (339/339), done.
remote: Total 466 (delta 159), reused 421 (delta 123), pack-reused 0
Receiving objects: 100% (466/466), 10.28 MiB | 20.92 MiB/s, done.
Resolving deltas: 100% (159/159), done.


In [8]:
class Boston():
  def __init__(self):
    # read
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep = "\s+", skiprows = 22, header = None)
    X = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    y = raw_df.values[1::2, 2]

    # split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = True, random_state = 142857)
    X_train = np.concatenate((X_train, y_train[:, None]), axis = 1)
    X_test  = np.concatenate((X_test, y_test[:, None]), axis = 1)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # noise
    X_train = add_train_random_noise(X_train, int(X_train.shape[0]*0.01))

    # 2d
    train_data = X_train[:, [0, 13]]
    test_data = X_test[:, [0, 13]]

    # flip
    train_data[:, 0] = 1 - train_data[:, 0]
    test_data[:, 0] = 1 - test_data[:, 0]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [9]:
class INTC_MSFT():
  def __init__(self):
    # read
    intel_f = open('gen-AC/data/raw/INTC_MSFT_GE/INTEL.data', 'r')
    intel = np.array(list(map(float, intel_f.readlines())))

    ms_f = open('gen-AC/data/raw/INTC_MSFT_GE/MS.data', 'r')
    ms = np.array(list(map(float, ms_f.readlines())))

    ge_f = open('gen-AC/data/raw/INTC_MSFT_GE/GE.data', 'r')
    ge = np.array(list(map(float, ge_f.readlines())))

    # split
    X = np.concatenate((intel[:, None], ms[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle = True, random_state = 142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

In [10]:
class GOOG_FB():
  def __init__(self):
    # read
    goog_f = open('gen-AC/data/raw/FB_GOOG/goog/close.vals', 'r')
    goog = np.array(list(map(float, goog_f.readlines())))

    fb_f = open('gen-AC/data/raw/FB_GOOG/fb/close.vals', 'r')
    fb = np.array(list(map(float, fb_f.readlines())))

    # split
    X = np.concatenate((goog[:, None], fb[:, None]), axis = 1)
    X_train, X_test, _, _ = train_test_split(X, X, shuffle=True, random_state=142857)

    # norm
    [X_train, X_test] = rank_normalization([X_train, X_test])

    # 2d, noise
    train_data = X_train[:, [0, 1]]
    train_data = add_train_random_noise(train_data, int(train_data.shape[0]*0.01))
    test_data = X_test[:, [0, 1]]

    self.train_y = train_data[:, 1].reshape(-1, 1)
    self.train_x = train_data[:, 0].reshape(-1, 1)
    self.validation_y = test_data[:, 1].reshape(-1, 1)
    self.validation_x = test_data[:, 0].reshape(-1, 1)

# params

In [11]:
batch_size = 128
max_epochs = 10000
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
device = logical_gpus[0].name

# boston

In [12]:
data_loader = Boston()

In [13]:
train_dataset = tf.data.Dataset.from_tensor_slices(
        ((data_loader.train_y, data_loader.train_x), np.zeros((len(data_loader.train_y), 0), dtype=np.float32)))
train_dataset = train_dataset.repeat(None)
train_dataset = train_dataset.shuffle(buffer_size=len(data_loader.train_y))
train_dataset = train_dataset.prefetch(3 * batch_size)
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices(
        ((data_loader.validation_y, data_loader.validation_x), np.zeros((len(data_loader.validation_y), 0), dtype=np.float32)))
val_dataset = val_dataset.repeat(1)
val_dataset = val_dataset.prefetch(len(data_loader.validation_y))
val_dataset = val_dataset.batch(len(data_loader.validation_y))

In [14]:
K.clear_session()
with tf.device(device):
#     model = Rnade(k_mix = 10, hidden_units=128, component_distribution="normal")
#     model = Rnade(k_mix = 10, hidden_units=128, component_distribution="laplace")
#     model = RnadeDeep(k_mix = 10, arch=[64,64], component_distribution="normal") # 6,366
#     model = RnadeDeep(k_mix = 10, arch=[64,64], component_distribution="laplace") # 6,366
#     model = MAF(num_bijectors=2, hidden_units=[32,32], covariate_hidden_units=[16,16], batch_norm=False) #5,188
#     model = MDN(arch=[64,64], num_mixtures=10) # 6,238
#     model = MONDE(cov_type="const_cov", arch_hxy=[64,64], arch_x_transform=[64,64], arch_cov_transform=None, hxy_x_size=32, covariance_learning_rate=None)
    model = MondeARMADE(arch = [32, 32], transform = 'sigm', x_transform_size = 16) # 2,241
    model.build([[None, 1], [None, 1]])
model.summary()

Model: "monde_armade"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 monde_armade_layer (MondeAR  multiple                 1185      
 MADELayer)                                                      
                                                                 
Total params: 1,185
Trainable params: 1,185
Non-trainable params: 0
_________________________________________________________________


In [15]:
def loss_fn(_, log_prob):
    return -tf.reduce_mean(log_prob)

model.compile(optimizer = tfk.optimizers.Adam(), loss = loss_fn)

os.makedirs('test', exist_ok = True)
model.save_to_json('test/{}_best_model_boston.json'.format(model.model_name))

callbacks = []
callbacks.append(tfk.callbacks.TerminateOnNaN())
callbacks.append(tfk.callbacks.ModelCheckpoint('test/{}_best_model_boston.h5'.format(model.model_name),
                                               monitor = 'val_loss', mode = 'min', verbose = 0, save_best_only = True, save_weights_only = True))
callbacks.append(tfk.callbacks.History())
# callbacks.append(tfk.callbacks.EarlyStopping(monitor = 'loss', patience = 30))

steps_per_epoch = int(np.ceil(len(data_loader.train_y) / batch_size))
with elapsed_timer("training"):
    history = model.fit(x = train_dataset, validation_data = val_dataset, verbose = 1, epochs = max_epochs, steps_per_epoch = steps_per_epoch, callbacks = callbacks)

# true_avg_ll = -np.mean(data_loader.ll(data_loader.validation_data))
fig = go.Figure()
fig.add_trace(go.Scatter(x = history.epoch, y = history.history['loss'], name = "train -ll"))
fig.add_trace(go.Scatter(x = history.epoch, y = history.history['val_loss'], name = "validation -ll"))
# fig.add_shape(go.layout.Shape(type = "line", xref = "paper", x0 = 0, y0 = true_avg_ll, x1 = 1, name = "true ll", y1 = true_avg_ll,
#                               line = dict(color = "LightSeaGreen", width = 4,dash = "dashdot")))
fig.update_layout(showlegend = True)
iplot(fig);

Streaming output truncated to the last 5000 lines.
3/3 [==============================] - 0s 15ms/step - loss: -0.2548 - val_loss: -0.2692
Epoch 7502/10000
3/3 [==============================] - 0s 17ms/step - loss: -0.3093 - val_loss: -0.2685
Epoch 7503/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.2557 - val_loss: -0.2689
Epoch 7504/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.3082 - val_loss: -0.2689
Epoch 7505/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.2408 - val_loss: -0.2682
Epoch 7506/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.2785 - val_loss: -0.2676
Epoch 7507/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.2447 - val_loss: -0.2671
Epoch 7508/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.3433 - val_loss: -0.2672
Epoch 7509/10000
3/3 [==============================] - 0s 13ms/step - loss: -0.2505 - val_loss: -0.2675
Epoch 7510/10000
3/3 

In [16]:
with tf.device(device):
  model = nl_utils.load_model_and_params('test/{}_best_model_boston'.format(model.model_name))

In [17]:
prob = model.prob(tf.convert_to_tensor(data_loader.validation_y, dtype = float), tf.convert_to_tensor(data_loader.validation_x, dtype = float), training = False).numpy()
yhat = -np.log(prob)
nll = np.mean(yhat)
nll

-0.2752004

In [18]:
res = bootstrap((yhat,), np.mean)
res.bootstrap_distribution.mean(), res.standard_error, res.confidence_interval

(-0.2755795,
 array([0.07030636], dtype=float32),
 ConfidenceInterval(low=array([-0.40040919]), high=array([-0.12396485])))

In [19]:
res = bootstrap((yhat,), np.mean)
rmean = res.bootstrap_distribution.mean()
low = res.confidence_interval.low
high = res.confidence_interval.high
dev1 = high - rmean
dev2 = rmean - low
dev = dev1 if dev1 > dev2 else dev2
rmean, res.standard_error, res.confidence_interval
print(f'NLL: {nll:.4f} RMean: {rmean:.4f} IC: ({low[0]:.4f},{high[0]:.4f}) Notation: {rmean:.4f}$\pm${dev[0]:.4f}')

NLL: -0.2752 RMean: -0.2764 IC: (-0.3997,-0.1202) Notation: -0.2764$\pm$0.1562


# intc-msft

In [20]:
data_loader = INTC_MSFT()

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices(
        ((data_loader.train_y, data_loader.train_x), np.zeros((len(data_loader.train_y), 0), dtype=np.float32)))
train_dataset = train_dataset.repeat(None)
train_dataset = train_dataset.shuffle(buffer_size=len(data_loader.train_y))
train_dataset = train_dataset.prefetch(3 * batch_size)
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices(
        ((data_loader.validation_y, data_loader.validation_x), np.zeros((len(data_loader.validation_y), 0), dtype=np.float32)))
val_dataset = val_dataset.repeat(1)
val_dataset = val_dataset.prefetch(len(data_loader.validation_y))
val_dataset = val_dataset.batch(len(data_loader.validation_y))

In [22]:
K.clear_session()
with tf.device(device):
#     model = Rnade(k_mix = 10, hidden_units=128, component_distribution="normal")
#     model = Rnade(k_mix = 10, hidden_units=128, component_distribution="laplace")
#     model = RnadeDeep(k_mix = 10, arch=[64,64], component_distribution="normal") # 6,366
#     model = RnadeDeep(k_mix = 10, arch=[64,64], component_distribution="laplace") # 6,366
#     model = MAF(num_bijectors=2, hidden_units=[32,32], covariate_hidden_units=[16,16], batch_norm=False) #5,188
#     model = MDN(arch=[64,64], num_mixtures=10) # 6,238
#     model = MONDE(cov_type="const_cov", arch_hxy=[64,64], arch_x_transform=[64,64], arch_cov_transform=None, hxy_x_size=32, covariance_learning_rate=None)
    model = MondeARMADE(arch = [32, 32], transform = 'sigm', x_transform_size = 16) # 2,241
    model.build([[None, 1], [None, 1]])
model.summary()

Model: "monde_armade"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 monde_armade_layer (MondeAR  multiple                 1185      
 MADELayer)                                                      
                                                                 
Total params: 1,185
Trainable params: 1,185
Non-trainable params: 0
_________________________________________________________________


In [23]:
def loss_fn(_, log_prob):
    return -tf.reduce_mean(log_prob)

model.compile(optimizer = tfk.optimizers.Adam(), loss = loss_fn)

os.makedirs('test', exist_ok = True)
model.save_to_json('test/{}_best_model_boston.json'.format(model.model_name))

callbacks = []
callbacks.append(tfk.callbacks.TerminateOnNaN())
callbacks.append(tfk.callbacks.ModelCheckpoint('test/{}_best_model_boston.h5'.format(model.model_name),
                                               monitor = 'val_loss', mode = 'min', verbose = 0, save_best_only = True, save_weights_only = True))
callbacks.append(tfk.callbacks.History())
# callbacks.append(tfk.callbacks.EarlyStopping(monitor = 'loss', patience = 30))

steps_per_epoch = int(np.ceil(len(data_loader.train_y) / batch_size))
with elapsed_timer("training"):
    history = model.fit(x = train_dataset, validation_data = val_dataset, verbose = 1, epochs = max_epochs, steps_per_epoch = steps_per_epoch, callbacks = callbacks)

# true_avg_ll = -np.mean(data_loader.ll(data_loader.validation_data))
fig = go.Figure()
fig.add_trace(go.Scatter(x = history.epoch, y = history.history['loss'], name = "train -ll"))
fig.add_trace(go.Scatter(x = history.epoch, y = history.history['val_loss'], name = "validation -ll"))
# fig.add_shape(go.layout.Shape(type = "line", xref = "paper", x0 = 0, y0 = true_avg_ll, x1 = 1, name = "true ll", y1 = true_avg_ll,
#                               line = dict(color = "LightSeaGreen", width = 4,dash = "dashdot")))
fig.update_layout(showlegend = True)
iplot(fig);

Streaming output truncated to the last 5000 lines.
8/8 [==============================] - 0s 11ms/step - loss: -0.1925 - val_loss: -0.1544
Epoch 7502/10000
8/8 [==============================] - 0s 9ms/step - loss: -0.2225 - val_loss: -0.1549
Epoch 7503/10000
8/8 [==============================] - 0s 12ms/step - loss: -0.2196 - val_loss: -0.1547
Epoch 7504/10000
8/8 [==============================] - 0s 13ms/step - loss: -0.2069 - val_loss: -0.1551
Epoch 7505/10000
8/8 [==============================] - 0s 13ms/step - loss: -0.2180 - val_loss: -0.1560
Epoch 7506/10000
8/8 [==============================] - 0s 14ms/step - loss: -0.2172 - val_loss: -0.1557
Epoch 7507/10000
8/8 [==============================] - 0s 12ms/step - loss: -0.1914 - val_loss: -0.1544
Epoch 7508/10000
8/8 [==============================] - 0s 12ms/step - loss: -0.1822 - val_loss: -0.1544
Epoch 7509/10000
8/8 [==============================] - 0s 11ms/step - loss: -0.2319 - val_loss: -0.1549
Epoch 7510/10000
8/8 [

In [24]:
with tf.device(device):
  model = nl_utils.load_model_and_params('test/{}_best_model_boston'.format(model.model_name))

In [25]:
prob = model.prob(tf.convert_to_tensor(data_loader.validation_y, dtype = float), tf.convert_to_tensor(data_loader.validation_x, dtype = float), training = False).numpy()
yhat = -np.log(prob)
nll = np.mean(yhat)
nll

-0.15793256

In [26]:
res = bootstrap((yhat,), np.mean)
res.bootstrap_distribution.mean(), res.standard_error, res.confidence_interval

(-0.15760867,
 array([0.03872387], dtype=float32),
 ConfidenceInterval(low=array([-0.23155587]), high=array([-0.08016542])))

In [27]:
res = bootstrap((yhat,), np.mean)
rmean = res.bootstrap_distribution.mean()
low = res.confidence_interval.low
high = res.confidence_interval.high
dev1 = high - rmean
dev2 = rmean - low
dev = dev1 if dev1 > dev2 else dev2
rmean, res.standard_error, res.confidence_interval
print(f'NLL: {nll:.4f} RMean: {rmean:.4f} IC: ({low[0]:.4f},{high[0]:.4f}) Notation: {rmean:.4f}$\pm${dev[0]:.4f}')

NLL: -0.1579 RMean: -0.1577 IC: (-0.2322,-0.0771) Notation: -0.1577$\pm$0.0806


# goog-fb

In [28]:
data_loader = GOOG_FB()

In [29]:
train_dataset = tf.data.Dataset.from_tensor_slices(
        ((data_loader.train_y, data_loader.train_x), np.zeros((len(data_loader.train_y), 0), dtype=np.float32)))
train_dataset = train_dataset.repeat(None)
train_dataset = train_dataset.shuffle(buffer_size=len(data_loader.train_y))
train_dataset = train_dataset.prefetch(3 * batch_size)
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices(
        ((data_loader.validation_y, data_loader.validation_x), np.zeros((len(data_loader.validation_y), 0), dtype=np.float32)))
val_dataset = val_dataset.repeat(1)
val_dataset = val_dataset.prefetch(len(data_loader.validation_y))
val_dataset = val_dataset.batch(len(data_loader.validation_y))

In [30]:
K.clear_session()
with tf.device(device):
#     model = Rnade(k_mix = 10, hidden_units=128, component_distribution="normal")
#     model = Rnade(k_mix = 10, hidden_units=128, component_distribution="laplace")
#     model = RnadeDeep(k_mix = 10, arch=[64,64], component_distribution="normal") # 6,366
#     model = RnadeDeep(k_mix = 10, arch=[64,64], component_distribution="laplace") # 6,366
#     model = MAF(num_bijectors=2, hidden_units=[32,32], covariate_hidden_units=[16,16], batch_norm=False) #5,188
#     model = MDN(arch=[64,64], num_mixtures=10) # 6,238
#     model = MONDE(cov_type="const_cov", arch_hxy=[64,64], arch_x_transform=[64,64], arch_cov_transform=None, hxy_x_size=32, covariance_learning_rate=None)
    model = MondeARMADE(arch = [32, 32], transform = 'sigm', x_transform_size = 16) # 2,241
    model.build([[None, 1], [None, 1]])
model.summary()

Model: "monde_armade"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 monde_armade_layer (MondeAR  multiple                 1185      
 MADELayer)                                                      
                                                                 
Total params: 1,185
Trainable params: 1,185
Non-trainable params: 0
_________________________________________________________________


In [31]:
def loss_fn(_, log_prob):
    return -tf.reduce_mean(log_prob)

model.compile(optimizer = tfk.optimizers.Adam(), loss = loss_fn)

os.makedirs('test', exist_ok = True)
model.save_to_json('test/{}_best_model_boston.json'.format(model.model_name))

callbacks = []
callbacks.append(tfk.callbacks.TerminateOnNaN())
callbacks.append(tfk.callbacks.ModelCheckpoint('test/{}_best_model_boston.h5'.format(model.model_name),
                                               monitor = 'val_loss', mode = 'min', verbose = 0, save_best_only = True, save_weights_only = True))
callbacks.append(tfk.callbacks.History())
# callbacks.append(tfk.callbacks.EarlyStopping(monitor = 'loss', patience = 30))

steps_per_epoch = int(np.ceil(len(data_loader.train_y) / batch_size))
with elapsed_timer("training"):
    history = model.fit(x = train_dataset, validation_data = val_dataset, verbose = 1, epochs = max_epochs, steps_per_epoch = steps_per_epoch, callbacks = callbacks)

# true_avg_ll = -np.mean(data_loader.ll(data_loader.validation_data))
fig = go.Figure()
fig.add_trace(go.Scatter(x = history.epoch, y = history.history['loss'], name = "train -ll"))
fig.add_trace(go.Scatter(x = history.epoch, y = history.history['val_loss'], name = "validation -ll"))
# fig.add_shape(go.layout.Shape(type = "line", xref = "paper", x0 = 0, y0 = true_avg_ll, x1 = 1, name = "true ll", y1 = true_avg_ll,
#                               line = dict(color = "LightSeaGreen", width = 4,dash = "dashdot")))
fig.update_layout(showlegend = True)
iplot(fig);

Streaming output truncated to the last 5000 lines.
8/8 [==============================] - 0s 11ms/step - loss: -1.0404 - val_loss: -1.0178
Epoch 7502/10000
8/8 [==============================] - 0s 9ms/step - loss: -0.9807 - val_loss: -1.0148
Epoch 7503/10000
8/8 [==============================] - 0s 10ms/step - loss: -1.0577 - val_loss: -1.0141
Epoch 7504/10000
8/8 [==============================] - 0s 11ms/step - loss: -1.0679 - val_loss: -1.0048
Epoch 7505/10000
8/8 [==============================] - 0s 11ms/step - loss: -1.0606 - val_loss: -1.0098
Epoch 7506/10000
8/8 [==============================] - 0s 12ms/step - loss: -1.1326 - val_loss: -1.0078
Epoch 7507/10000
8/8 [==============================] - 0s 11ms/step - loss: -0.9916 - val_loss: -1.0066
Epoch 7508/10000
8/8 [==============================] - 0s 13ms/step - loss: -1.0450 - val_loss: -1.0143
Epoch 7509/10000
8/8 [==============================] - 0s 8ms/step - loss: -1.0462 - val_loss: -1.0114
Epoch 7510/10000
8/8 [=

In [32]:
with tf.device(device):
  model = nl_utils.load_model_and_params('test/{}_best_model_boston'.format(model.model_name))

In [33]:
prob = model.prob(tf.convert_to_tensor(data_loader.validation_y, dtype = float), tf.convert_to_tensor(data_loader.validation_x, dtype = float), training = False).numpy()
yhat = -np.log(prob)
nll = np.mean(yhat)
nll

-1.0906385

In [34]:
res = bootstrap((yhat,), np.mean)
res.bootstrap_distribution.mean(), res.standard_error, res.confidence_interval

(-1.090559,
 array([0.0609326], dtype=float32),
 ConfidenceInterval(low=array([-1.20730362]), high=array([-0.96814901])))

In [35]:
res = bootstrap((yhat,), np.mean)
rmean = res.bootstrap_distribution.mean()
low = res.confidence_interval.low
high = res.confidence_interval.high
dev1 = high - rmean
dev2 = rmean - low
dev = dev1 if dev1 > dev2 else dev2
rmean, res.standard_error, res.confidence_interval
print(f'NLL: {nll:.4f} RMean: {rmean:.4f} IC: ({low[0]:.4f},{high[0]:.4f}) Notation: {rmean:.4f}$\pm${dev[0]:.4f}')

NLL: -1.0906 RMean: -1.0916 IC: (-1.2082,-0.9638) Notation: -1.0916$\pm$0.1278
